In [ ]:
import sys
sys.path.append('../input/mytimm/pytorch-image-models-master')
from timm import create_model

import PIL
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator,array_to_img
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

from fastai.vision.all import *

set_seed(2999, reproducible=True)
BATCH_SIZE = 16    

In [ ]:
dataset_path = Path('../input/petfinder-pawpularity-score/')

seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/myswin/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'


In [ ]:
test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()

In [ ]:
tlen = len(test_df)
tlen

In [ ]:
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
#test_df = test_df.drop(columns=['Id'])

In [ ]:
test_ptypes = []
paths = list(test_df['path'].values) 

In [ ]:
import gc
learn = load_learner('../input/catdog384/cat_dog384.pkl', cpu=False)
for tid in paths:
    test_ptypes.append((learn.predict(tid))[1].item())
print(test_ptypes)

In [ ]:
del learn
torch.cuda.empty_cache()
gc.collect()

In [ ]:
test_df['pets'] = test_ptypes
testD_df = test_df[test_df.pets>0]
len(testD_df)

In [ ]:
testC_df = test_df[test_df.pets==0]
len(testC_df)

In [ ]:
modelA_dir = '../input/models7f/models7/'
modelC_dir = '../input/my7cats/'
modelD_dir = '../input/my7dogs/'
modelC1_dir = '../input/my7cats1/'
modelC2_dir = '../input/fastai-double/'
modelD1_dir = '../input/my7dogs1/'
modelD2_dir = '../input/my7dogs2/'
modelT_dir = '../input/models12e6/'
#modelP_dir = '../input/models7full13/'

def petfinder_rmse( input, target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
dls = ImageDataLoaders.from_df(test_df, seed=999, fn_col='path', y_block=RegressionBlock, 
                               bs=BATCH_SIZE, num_workers=8, item_tfms=Resize(224), 
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()]))

test_dl = dls.test_dl(test_df)

In [ ]:
import gc
N_FOLDS =7
all_predsA = []
for i in range(N_FOLDS):
    print(f'Fold {i} results')
    test_dl = dls.test_dl(test_df)
    learn = load_learner(modelA_dir+f'model_fold_{i}.pkl', cpu=False)   
    learn.dls.to('cuda')
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsA.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import gc

FOLDS = [0, 2, 3, 5, 6, 11]
all_predsT = []

for fd in FOLDS:
    print(f'Fold {fd} results')
    test_dl = dls.test_dl(test_df)
    learn = load_learner(modelT_dir+f'model_fold_{fd}.pkl', cpu=False)   
    learn.dls.to('cuda')
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsT.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
dls = ImageDataLoaders.from_df(testC_df, seed=999, fn_col='path', y_block=RegressionBlock, 
                               bs=BATCH_SIZE, num_workers=8, item_tfms=Resize(224), 
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()]))

In [ ]:
import gc
N_FOLDS = 7
all_predsC = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')
    test_dl = dls.test_dl(testC_df)
    learn = load_learner(modelC_dir+f'modelC_fold_{i}.pkl', cpu=False)   
    learn.dls.to('cuda')
    #learn = learn.to_fp16()
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsC.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import gc
N_FOLDS = 7
all_predsC1 = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')
    test_dl = dls.test_dl(testC_df)
    learn = load_learner(modelC1_dir+f'modelC_fold_{i}.pkl', cpu=False)   
    learn.dls.to('cuda')
    #learn = learn.to_fp16()
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsC1.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import gc
N_FOLDS = 7
all_predsC2 = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')
    test_dl = dls.test_dl(testC_df)
    learn = load_learner(modelC2_dir+f'modelC_fold_{i}.pkl', cpu=False)   
    learn.dls.to('cuda')
    #learn = learn.to_fp16()
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsC2.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
dls = ImageDataLoaders.from_df(testD_df, seed=999, fn_col='path', y_block=RegressionBlock, 
                               bs=BATCH_SIZE, num_workers=8, item_tfms=Resize(224), 
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()]))

In [ ]:
import gc
N_FOLDS = 7
all_predsD = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')
    test_dl = dls.test_dl(testD_df)
    learn = load_learner(modelD_dir+f'modelD_fold_{i}.pkl', cpu=False)   
    learn.dls.to('cuda')
    #learn = learn.to_fp16()
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsD.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import gc
N_FOLDS = 7
all_predsD1 = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')
    test_dl = dls.test_dl(testD_df)
    learn = load_learner(modelD1_dir+f'modelD_fold_{i}.pkl', cpu=False)   
    learn.dls.to('cuda')
    #learn = learn.to_fp16()
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsD1.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import gc
N_FOLDS = 7
all_predsD2 = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')
    test_dl = dls.test_dl(testD_df)
    learn = load_learner(modelD2_dir+f'modelD_fold_{i}.pkl', cpu=False)   
    learn.dls.to('cuda')
    #learn = learn.to_fp16()
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_predsD2.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
np.mean(np.stack(all_predsA*100))

In [ ]:
np.mean(np.stack(all_predsT*100))

In [ ]:
np.mean(np.stack(all_predsC*100))

In [ ]:
np.mean(np.stack(all_predsC1*100))

In [ ]:
np.mean(np.stack(all_predsC2*100))

In [ ]:
np.mean(np.stack(all_predsD*100))

In [ ]:
np.mean(np.stack(all_predsD1*100))

In [ ]:
np.mean(np.stack(all_predsD2*100))

In [ ]:
predsC = np.mean(np.stack(all_predsC), axis=0)
predsD = np.mean(np.stack(all_predsD), axis=0)
predsC1 = np.mean(np.stack(all_predsC1), axis=0)
predsC2 = np.mean(np.stack(all_predsC2), axis=0)
predsD1 = np.mean(np.stack(all_predsD1), axis=0)
predsD2 = np.mean(np.stack(all_predsD2), axis=0)

In [ ]:
#testC_df['Id'] = testC_df['path'].apply(lambda pp: pp.split('/')[1].split('.')[0])
testC_df['pawsC'] = predsC*100
testC_df['pawsC1'] = predsC1*100
testC_df['pawsC2'] = predsC2*100
ctk = ['Id', 'pawsC', 'pawsC1', 'pawsC2']
testC_df = testC_df[ctk]
#testC_df

In [ ]:
#testD_df['Id'] = testD_df['path'].apply(lambda pp: pp.split('/')[1].split('.')[0])
testD_df['pawsD'] = predsD*100
testD_df['pawsD1'] = predsD1*100
testD_df['pawsD2'] = predsD2*100
ctk = ['Id', 'pawsD', 'pawsD1', 'pawsD2']
testD_df = testD_df[ctk]
#testD_df

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
predsA = np.mean(np.stack(all_predsA), axis=0)
#predsP = np.mean(np.stack(all_predsP), axis=0)
predsT = np.mean(np.stack(all_predsT), axis=0)
#sample_df['category'] = np.round(preds*12)
sample_df['pawsA'] = predsA*100
sample_df['pawsT'] = predsT*100

In [ ]:
sample_df = pd.merge (sample_df, testC_df, on='Id', how='outer')
sample_df = pd.merge (sample_df, testD_df, on='Id', how='outer')
sample_df.fillna(0, inplace=True)
sample_df['Cpaws'] = (sample_df['pawsC']+sample_df['pawsC1']+sample_df['pawsC2'])/3
sample_df['Dpaws'] = (sample_df['pawsD']+sample_df['pawsD1']+sample_df['pawsD1'])/3
sample_df['Pawpularity']=(sample_df['pawsA']+sample_df['pawsT']+3*sample_df['Cpaws']+3*sample_df['Dpaws'])/5

del sample_df['Cpaws']
del sample_df['Dpaws']
del sample_df['pawsC']
del sample_df['pawsC1']
del sample_df['pawsC2']
del sample_df['pawsD']
del sample_df['pawsD1']
del sample_df['pawsD2']
del sample_df['pawsA']
del sample_df['pawsT']

In [ ]:
sample_df.to_csv('submission.csv',index=False)
pd.read_csv('submission.csv')